# Quickstart tutorial
<br>
<br>
## Prerequisites
----------------------------
### Install
Please install package PyMCEF through eighter conda or pip:

<pre>
$ conda install -c hzzyyy pymcef

$ pip install pymcef
</pre>

conda packages are available for both python 2.7 and 3.5 on MacOS, Linux and Windows.

pip packages are available for both python 2.7 and 3.5 on MacOs, Windows but not Linux.

This package is only available on 64 bits OS.

### Efficient frontier
This package helps to find the efficient frontier based on (jointly) simulated returns of all investible assets.

Alternatively speaking, this package tries to solve this problem numerically:

> Given the joint distribution of the returns of all the investible assets, what should be the
> best choice of weights to assign on each assets, for all possible risk preferences.

Efficient frontier is the set of all 'best' choices of asset weights (a.k.a. portfolio) with all possible risk preferences.

Heuristically, the most risk-taking portfolio simply puts all weight on the asset with largest expected return. 

The most risk-averse portfolio is very diversified, with many positions hedging each other.

### Portfolio optimization

Each portfolio on the efficient frontier is obtained by solving the following optimization problem on the choice of weight vector $w$:

\begin{align*}
\underset{w}{\mathrm{argmin}}\quad & \mathrm{Risk}\left(w\right)-\lambda\cdot\mathrm{Reward}\left(w\right),\\
\mathrm{subject\ to}\quad & \sum_{i}w_{i} = 1,\\
 & \ \ \ \quad w_{i} > 0.
\end{align*}

The Lagrangian multiplier $\lambda$ here uniquely defines a risk preference and its corresponding solution. 
With $\lambda$ going to infinity the solution is the most risk-taking portfolio and with $\lambda$ being zero, the solution is the most risk-averse portfolio.

### Risk measures

We haven't defined the formulas for the risk and reward functions in the above optimization problem. Not surprisingly, the reward function is the just the expected return of the whole portfolio. Suppose the returns of all the asserts is the random vector: 

$$ Y = \{Y_i\}. $$ 

Then the reward function is simply:

\begin{eqnarray*}
\mathrm{Reward}\left(w\right) & = & \mathbb{E}\left[\sum_{i}w_{i}Y_{i}\right]\\
 & = & \mathbb{E}\left[X\right]\quad\mathrm{namely}
\end{eqnarray*}

The less obviously obtained risk function, formally known as risk measure, is a function of the portfolio return random variable $X$. 
In the Markowitz mean-variance model, the risk measure is the variance of $X$. In this PyMCEF, the following two more sophisticated risk measures are used:

####  Absolute Semideviation

\begin{eqnarray*}
\mathrm{Risk}\left(w\right) & = & \mathbb{E}\left[\max\left(\mathbb{E}X-X,\ 0\right)\right]\\
 & = & \mathbb{E}\left[\left(\mathbb{E}X-X\right)^{+}\right].
\end{eqnarray*}

#### Fixed-target under-performance

\begin{eqnarray*}
\mathrm{Risk}\left(w\right) & = & \mathbb{E}\left[\max\left(t-X,\ 0\right)\right]\\
 & = & \mathbb{E}\left[\left(t-X\right)^{+}\right],
\end{eqnarray*}

where $t$ is given target. The riskless return is a sensible choice. 

These two risk measures are more sophisticated than the variance because they are more consistent with the stochastic dominance theory. 

### Stochastic programming

The (joint) distribution of the returns of assets $\{Y_i\}$ can be parametric. A convenient
choice is to assume Normal or log Normal distribution. More complicated ways of
parameterizations for sure exist, and they lead to different optimization problems. 
In addition with $\lambda$ varying as a positive real number, we face a continuum of
optimization problems.

Alternatively, we can work with the (finite) samples of $\{Y_i\}$.
In this case, we replace the expectation in the risk or reward function with statistical mean and work with a stochastic programming problem. 
PyMCEF is implemented in this way, and there is huge advantage in terms of flexibility. 
The input for PyMCEf is just Monte Carlo simulated returns, and no knowledge about the underlying distribution function is needed for this package to work. 
In Bayesian inference, it is a common practice to directly simulate the posterior predictive
distribution with Markov Chain Monte Carlo, where the posterior density function is known only 
up to a scaling factor. In another word, in practice, samples of distribution is more available than a parametric form.


## One example
---------